In [1]:
import sys
sys.path.append("../gaia_tools/")
sys.path.append("../scripts/")
from covariance_generation import generate_galactocentric_covmat, transform_cov_cylindirical
import covariance_generation
from mcmc_plots import *
import data_analysis
import transformation_constants
from velocity_curve_plots import load_bin_settings
import helper_functions as helpfunc
import numpy as np
import emcee
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import pickle
from pylab import *
from scipy.optimize import curve_fit
from cProfile import label
import pandas as pd
import pickle


In [13]:
def load_galactic_parameters():
   '''The load_galactic_parameters function sets the initial galactocentric distance, height over the Galactic plane, and solar vector used in the coordinate transformations.'''

   # Initial Galactocentric distance
   r_0 = 8277

   # Initial height over Galactic plane
   z_0 = 25

   # Initial solar vector
   v_sun = transformation_constants.V_SUN
   v_sun[0][0] = 11.1
   v_sun[1][0] = 251.5
   v_sun[2][0] = 8.59

   # Eilers et al orbital parmaeters
   # R0 - 8122
   # z0 - 25
   # v_sun[0][0] = 11.1
   # v_sun[1][0] = 245.8
   # v_sun[2][0] = 7.8

   return r_0, z_0, v_sun

def apply_initial_cut(icrs_data):

   r_0, z_0, v_sun = load_galactic_parameters()

   galcen_data = data_analysis.get_transformed_data(icrs_data,
                                          include_cylindrical = True,
                                          z_0 = z_0,
                                          r_0 = r_0,
                                          v_sun = v_sun,
                                          debug = True,
                                          is_bayes = True,
                                          is_source_included = True)

   print("Galactocentric data shape: {}".format(galcen_data.shape))

   galactocentric_cov = covariance_generation.generate_galactocentric_covmat(icrs_data, 
                                                               is_bayes = True,
                                                               Z_0 = z_0,
                                                               R_0 = r_0)

   cyl_cov = covariance_generation.transform_cov_cylindirical(galcen_data, 
                                                C = galactocentric_cov,
                                                Z_0 = z_0,
                                                R_0 = r_0)
   galcen_data = galcen_data.merge(cyl_cov, on='source_id')

   print("Galactocentric data shape after merge with covariance info: {}".format(galcen_data.shape))

   # Remove noisy distances
   print("Removing noisy distances")
   galcen_data['parallax_over_error'] = icrs_data.parallax_over_error[galcen_data.source_id == icrs_data.source_id]
   galcen_data = galcen_data[galcen_data.parallax_over_error > 8]
   galcen_data = galcen_data.drop(columns=['parallax_over_error'])

   print("Galactocentric data shape after removing noisy distances: {}".format(galcen_data.shape))

   # Final data cut
   galcen_data = galcen_data[(galcen_data.r < 15000) & (galcen_data.r > 5000)]
   galcen_data = galcen_data[(galcen_data.z < 200) & (galcen_data.z > -200)]

   print("Galactocentric data shape after constraining region: {}".format(galcen_data.shape))

   # Remove halo stars (condition taken from 1806.06038)                        
   v_dif = np.linalg.norm(np.array([galcen_data.v_x, galcen_data.v_y, galcen_data.v_z])-v_sun,
                        axis=0)                                               
   galcen_data['v_dif'] = v_dif                                                 
   galcen_data = galcen_data[galcen_data.v_dif<210.]

   print("Galactocentric data shape after removing halo stars: {}".format(galcen_data.shape))

   galcen_data.reset_index(inplace=True, drop=True)
   
   return galcen_data

In [3]:
print('Importing DR3...')
dr3_path = '/local/mariacst/2022_v0_project/data/GaiaDR3_RV_RGB_fidelity.csv'
gaia_dr3 = pd.read_csv(dr3_path)

r_est_error = (gaia_dr3.B_rpgeo - gaia_dr3.b_rpgeo)/2
gaia_dr3['r_est_error'] = r_est_error

parallax_over_error = gaia_dr3.parallax/gaia_dr3.parallax_error
gaia_dr3['parallax_over_error'] = parallax_over_error

columns_to_drop = ['Vbroad', 'GRVSmag', 'Gal', 'Teff', 'logg',
    '[Fe/H]', 'Dist', 'A0', 'RAJ2000', 'DEJ2000', 'e_RAJ2000', 'e_DEJ2000',
    'RADEcorJ2000', 'B_Teff', 'b_Teff', 'b_logg', 'B_logg', 'b_Dist',
    'B_Dist', 'b_AG', 'B_AG', 'b_A0', 'B_A0', 'Gmag', 'BPmag', 'RPmag', 'BP-RP']
gaia_dr3 = gaia_dr3.drop(columns=columns_to_drop)
print(gaia_dr3.columns)


Importing DR3...
Index(['Unnamed: 0', 'source_id', 'ra', 'dec', 'rgeo', 'b_rgeo', 'B_rgeo',
       'r_est', 'b_rpgeo', 'B_rpgeo', 'ra_error', 'dec_error', 'parallax',
       'parallax_error', 'PM', 'pmra', 'pmra_error', 'pmdec', 'pmdec_error',
       'radial_velocity', 'radial_velocity_error', 'ra_dec_corr',
       'ra_pmdec_corr', 'ra_parallax_corr', 'ra_pmra_corr', 'dec_pmra_corr',
       'dec_pmdec_corr', 'dec_parallax_corr', 'parallax_pmdec_corr',
       'parallax_pmra_corr', 'pmra_pmdec_corr', 'r_est_error',
       'parallax_over_error'],
      dtype='object')


In [4]:
icrs_data = gaia_dr3

In [33]:
r_0, z_0, v_sun = load_galactic_parameters()

galcen_data = apply_initial_cut(icrs_data)


Starting galactocentric transformation loop over all data points.. 
Time elapsed for data coordinate transformation: 7.393428396433592 sec
Galactocentric data shape: (6586329, 11)
Galactocentric data shape after merge with covariance info: (6586329, 13)
Removing noisy distances
Galactocentric data shape after removing noisy distances: (4264749, 13)
Galactocentric data shape after constraining region: (1461488, 13)
Galactocentric data shape after removing halo stars: (1458250, 14)


In [34]:
r_min = 5000/8277
r_max = 15000/8277

# Generate bins
bin_collection = data_analysis.get_collapsed_bins(data = galcen_data,
                                                        theta = 8277,
                                                        BL_r_min = r_min,
                                                        BL_r_max = r_max,
                                                        BL_z_min = -200,
                                                        BL_z_max = 200,
                                                        N_bins = (10, 1),
                                                        r_drift = True,
                                                        debug = False)

In [6]:
bin1, bin2 = [bin_collection.bins[-2], bin_collection.bins[-1]]

NameError: name 'bin_collection' is not defined

In [44]:
del bin_collection.bins[-2]
del bin_collection.bins[-1]
bin_collection.bins

In [53]:
# Get bin number of the bin from which the merging starts
new_bin_num = bin1.bin_num
merged_dataset = pd.concat([bin1.data, bin2.data], ignore_index=True)
merged_dataset['Bin_index'] = new_bin_num

In [65]:
print(bin1.r_boundaries)
print(bin2.r_boundaries)

# Merge bounadries
new_r_boundaries = (bin1.r_boundaries[0], bin2.r_boundaries[-1])
print(new_r_boundaries)

# Same as before
new_z_boundaries = bin1.z_boundaries
print(new_z_boundaries)

(1.5706173734444846, 1.6914340944786757)
(1.6914340944786757, 1.812250815512867)
(1.5706173734444846, 1.812250815512867)
(-200.0, 200.0)


In [70]:
from Bin import Bin
merged_bin = Bin(merged_dataset)
merged_bin.r_boundaries = new_r_boundaries
merged_bin.z_boundaries = new_z_boundaries

In [71]:
print(len(merged_bin.data))
print(merged_bin.r_boundaries)
print(merged_bin.z_boundaries)

2127
(1.5706173734444846, 1.812250815512867)
(-200.0, 200.0)


In [73]:
# Add to bincollection
bin_collection.bins.append(merged_bin)

In [118]:
bin_collection.data

,x,y,z,v_x,v_y,v_z,r,phi,v_r,v_phi,source_id,sig_vphi,sig_vr,v_dif,r_orig,Bin_index
0,-8392.695681,8.087171,-103.851431,82.402369,174.493940,38.347357,1.013978,3.140629,-82.234189,-174.573262,132667245587072,0.060109,0.013249,109.084652,8392.699577,13
1,-8417.299711,8.997379,-126.899476,-21.557459,207.624678,6.182526,1.016951,3.140524,21.779379,-207.601516,250246270117248,0.049664,0.015947,54.748054,8417.304519,13
2,-8378.115185,8.745594,-86.468071,30.519979,255.173686,-5.822975,1.012217,3.140549,-30.253597,-255.205405,454514914751488,0.029568,0.829964,24.461508,8378.119749,13
3,-8431.833054,15.299558,-144.698540,6.319658,226.135774,-12.278258,1.018708,3.139778,-5.909325,-226.146869,1301929142132608,0.030242,0.008581,33.191562,8431.846935,13
4,-8452.841025,15.246163,-141.687048,-0.931497,250.678662,-14.943075,1.021246,3.139789,1.383638,-250.676574,3188656735589888,0.000970,0.010729,26.443093,8452.854775,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1614918,-8139.981263,148.009181,-92.915042,46.237269,223.187107,11.631007,0.983608,3.123412,-42.172090,-223.990814,6916181546077871744,0.022511,0.025569,45.227152,8141.326776,13
1614919,-8100.135599,187.748608,-123.633054,22.567053,250.557152,-0.468166,0.978895,3.118418,-16.755026,-251.012803,6916198691587462272,2.014342,1.625323,14.643519,8102.311168,13
1614920,-8090.595920,203.266898,-128.033077,48.913031,221.128029,-11.587390,0.977788,3.116474,-43.343767,-222.286769,6917092972496901504,0.092599,0.048159,52.530077,8093.148941,13
1614921,-8097.911973,195.069190,-116.886433,24.541230,263.888796,5.405961,0.978647,3.117508,-18.179185,-264.403264,6917335410515389696,0.007294,0.007852,18.554973,8100.261126,13


In [121]:
len(bin1.data) + len(bin2.data)

2127

In [129]:
bin_collection.bins[-2].data

,x,y,z,v_x,v_y,v_z,r,phi,v_r,v_phi,source_id,sig_vphi,sig_vr,v_dif,r_orig,Bin_index
3638,-13042.074428,428.864107,-195.235138,-23.430872,228.014679,12.559298,1.576552,3.108721,30.911998,-227.121439,180718759097550720,4.767174,1.038036,41.948740,13049.123718,28
3667,-13119.640373,448.384833,-194.515122,8.387728,238.605280,9.706469,1.585997,3.107429,-0.232873,-238.752549,180746865363492096,1.254986,1.508094,13.224096,13127.300273,28
3750,-13523.724807,524.276454,-146.790558,-2.020394,226.880586,-6.400216,1.635119,3.102845,10.807792,-226.632023,180879322155542656,3.027300,3.290122,31.669652,13533.883348,28
3897,-13112.006997,552.755035,-167.542355,0.458190,229.578025,-4.141839,1.585557,3.099461,9.211812,-229.393596,181057374318242688,4.857008,58.689044,27.494015,13123.652907,28
4068,-13047.655780,593.848950,-111.662249,-12.647921,242.617891,6.119846,1.578007,3.096110,23.665894,-241.791928,181256317203599104,2.155393,13.105844,25.474640,13061.162963,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1150510,-11916.834705,-5624.472823,162.740352,-103.306159,213.438985,-1.193313,1.592058,-2.700614,2.322195,-237.113834,5715298038607706368,38.495965,7.637779,120.967447,13177.467281,28
1150526,-12090.486766,-5877.077813,194.449446,-87.055228,190.949453,-1.020528,1.624165,-2.689134,-5.183636,-209.793794,5715306869060269440,33.085087,19.580774,115.728907,13443.210697,28
1150792,-11878.753618,-5436.593648,65.911986,-85.941363,195.486009,2.265271,1.578318,-2.712376,-3.207483,-213.519109,5715435030884292096,13.068900,7.145845,112.225645,13063.741348,28
1150918,-12263.396177,-6050.472118,169.495451,-100.248855,191.953620,-3.642084,1.652140,-2.683258,4.971305,-216.497832,5715509900751789184,24.142576,8.470617,126.861984,13674.761374,28


In [130]:
len(bin_collection.bins[-2].data) + len(bin_collection.bins[-1].data)

2127

In [133]:
len(bin_collection.data)

1614923

In [136]:
# Before merge
data_length = 0 
for bin in bin_collection.bins:
    print(bin.bin_num)
    print(len(bin.data))
    data_length += len(bin.data)

data_length

4
258999
7
486074
10
366930
13
213543
16
145586
19
92696
22
37884
25
11083
28
1951
31
176


1614922

In [119]:
data_length = 0 
for bin in bin_collection.bins:
    data_length += len(bin.data)

data_length

1614922

In [35]:
print(bin_collection.bin_num_set)
for bin in bin_collection.bins:
    print(len(bin.data))
    print(bin.r_boundaries)

{4, 7, 10, 13, 16, 18, 19, 22, 25, 28, 31}
187758
(0.6040836051709556, 0.7249003262051468)
446262
(0.7249003262051468, 0.8457170472393378)
349219
(0.8457170472393378, 0.966533768273529)
206739
(0.966533768273529, 1.0873504893077202)
141481
(1.0873504893077202, 1.2081672103419114)
89436
(1.2081672103419114, 1.3289839313761025)
31828
(1.3289839313761025, 1.4498006524102935)
5211
(1.4498006524102935, 1.5706173734444846)
308
(1.5706173734444846, 1.6914340944786757)
7
(1.6914340944786757, 1.812250815512867)


In [21]:
bin_collection.merge_bins([-2, -1])

(1.3289839313761025, 1.812250815512867)
(-200.0, 200.0)


In [22]:
print(bin_collection.bin_num_set)
for bin in bin_collection.bins:
    print(len(bin.data))
    print(bin.r_boundaries)

{4, 7, 10, 13, 16, 19, 22}
187758
(0.6040836051709556, 0.7249003262051468)
446262
(0.7249003262051468, 0.8457170472393378)
349219
(0.8457170472393378, 0.966533768273529)
206739
(0.966533768273529, 1.0873504893077202)
141481
(1.0873504893077202, 1.2081672103419114)
89436
(1.2081672103419114, 1.3289839313761025)
37354
(1.3289839313761025, 1.812250815512867)
